# 02. FP8 양자화 — EXAONE-4.0-1.2B

**전략:** 평가 서버가 L4 GPU인데, L4는 FP8 연산을 하드웨어 레벨로 가속함.  
GPTQ W4A16보다 **추론 속도가 더 빠를 수 있는** 핵심 차별화 기법.

| 방식 | 가중치 | 활성화 | L4 하드웨어 가속 | 예상 효과 |
|------|--------|--------|-----------------|----------|
| GPTQ W4A16 (베이스라인) | 4bit | 16bit | ❌ | 속도 보통 |
| **FP8 W8A8** | 8bit | 8bit | ✅ INT8/FP8 텐서코어 | **속도 최강** |

**환경:** Google Colab Free (T4 16GB)  
**vLLM:** 0.14.1 (FP8 compressed-tensors 포맷 지원)

## 0. 환경 확인

In [ ]:
import torch
print(f"PyTorch: {torch.__version__}")
print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

# FP8 지원 여부 확인 (Ampere 이상: A100, L4, H100 등)
# T4는 FP8 미지원이지만 캘리브레이션/저장은 가능
# → 평가 서버(L4)에서 실제 FP8 하드웨어 가속 실행됨
compute_cap = torch.cuda.get_device_capability()
print(f"Compute Capability: {compute_cap}")
if compute_cap[0] >= 8:
    print("✅ FP8 하드웨어 연산 지원 (이 GPU에서 FP8 추론 가능)")
else:
    print("⚠️  이 GPU(T4)는 FP8 하드웨어 연산 미지원")
    print("   → 양자화/저장은 여기서, 실제 FP8 가속은 평가 서버(L4)에서 동작")

## 1. 패키지 설치

> ⚠️ 설치 후 **런타임 재시작** 필요

In [ ]:
!pip install -q \
    llmcompressor \
    "compressed-tensors==0.13.0" \
    "transformers>=4.54.0" \
    datasets \
    accelerate \
    sentencepiece

print("\n✅ 설치 완료! 런타임을 재시작하세요")

## 2. 설정값

In [ ]:
# ============================================================
# FP8 실험 설정
# ============================================================

MODEL_ID = "LGAI-EXAONE/EXAONE-4.0-1.2B"
OUT_DIR  = "./model_fp8"

DATASET_ID    = "LGAI-EXAONE/MANTA-1M"
DATASET_SPLIT = "train"

NUM_CALIBRATION_SAMPLES = 512   # FP8은 상대적으로 샘플 수에 덜 민감
MAX_SEQUENCE_LENGTH     = 1024

# FP8 W8A8: 가중치 FP8 + 활성화 FP8 → L4에서 하드웨어 가속
SCHEME  = "FP8"
TARGETS = ["Linear"]
IGNORE  = ["embed_tokens", "lm_head"]  # 임베딩 레이어는 FP8 제외

ZIP_NAME = "submit_fp8"

print("=" * 50)
print(f"모델        : {MODEL_ID}")
print(f"양자화 방식 : {SCHEME} (W8A8, L4 하드웨어 가속)")
print(f"캘리브 샘플 : {NUM_CALIBRATION_SAMPLES}")
print(f"시퀀스 길이 : {MAX_SEQUENCE_LENGTH}")
print(f"저장 경로   : {OUT_DIR}")
print("=" * 50)

## 3. 모델 & 토크나이저 로드

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

print("토크나이저 로드 중...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)

print("모델 로드 중...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
)
print("✅ 로드 완료")

## 4. 캘리브레이션 데이터 준비

In [ ]:
from datasets import load_dataset

print(f"데이터 로드: {NUM_CALIBRATION_SAMPLES}샘플")
ds = load_dataset(DATASET_ID, split=f"{DATASET_SPLIT}[:{NUM_CALIBRATION_SAMPLES}]")

def preprocess(example):
    return {
        "text": tokenizer.apply_chat_template(
            example["conversations"],
            add_generation_prompt=True,
            tokenize=False,
        )
    }

ds = ds.map(preprocess)
print(f"✅ 데이터 준비: {len(ds)}개")

## 5. FP8 양자화 실행

> T4에서 약 10~20분 소요 (GPTQ보다 빠름)

In [ ]:
from llmcompressor import oneshot
from llmcompressor.modifiers.quantization import QuantizationModifier
import time

print(f"FP8 양자화 시작 (scheme={SCHEME})...")
start_time = time.time()

# FP8은 GPTQModifier 대신 QuantizationModifier 사용
recipe = [
    QuantizationModifier(
        targets=TARGETS,
        scheme=SCHEME,
        ignore=IGNORE,
    )
]

oneshot(
    model=model,
    dataset=ds,
    recipe=recipe,
    max_seq_length=MAX_SEQUENCE_LENGTH,
    num_calibration_samples=NUM_CALIBRATION_SAMPLES,
)

elapsed = time.time() - start_time
print(f"\n✅ FP8 완료! 소요 시간: {elapsed/60:.1f}분")

## 6. 저장 & 제출 파일 생성

In [ ]:
import os, shutil
from pathlib import Path

os.makedirs(OUT_DIR, exist_ok=True)
print(f"모델 저장 중: {OUT_DIR}")

model.save_pretrained(OUT_DIR, save_compressed=True)
tokenizer.save_pretrained(OUT_DIR)

# 파일 크기 확인
total_size = sum(f.stat().st_size for f in Path(OUT_DIR).iterdir()) / 1024**3
print(f"\n총 크기: {total_size:.2f} GB")
print("(FP8은 BF16 대비 약 50% 크기, GPTQ W4A16보다는 큼)")

# config.json의 quantization_config 확인 (vLLM이 이걸 읽어서 FP8 커널 선택)
import json
with open(f"{OUT_DIR}/config.json") as f:
    cfg = json.load(f)
print(f"\nquantization_config: {cfg.get('quantization_config', 'None')}")

print("\n✅ 저장 완료")

In [ ]:
# 제출용 zip
shutil.make_archive(base_name=ZIP_NAME, format="zip", root_dir=".", base_dir=OUT_DIR)
zip_size = Path(f"{ZIP_NAME}.zip").stat().st_size / 1024**3
print(f"✅ {ZIP_NAME}.zip 생성 완료 ({zip_size:.2f} GB)")

## 7. Sanity Check

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

print("저장된 FP8 모델 로드 및 추론 테스트...")
model_check = AutoModelForCausalLM.from_pretrained(
    OUT_DIR,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
)
tok_check = AutoTokenizer.from_pretrained(OUT_DIR, trust_remote_code=True)

messages = [{"role": "user", "content": "대한민국의 수도는"}]
inputs = tok_check.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model_check.device)

with torch.no_grad():
    outputs = model_check.generate(inputs, max_new_tokens=50, do_sample=False)

response = tok_check.decode(outputs[0][inputs.shape[-1]:], skip_special_tokens=True)
print(f"모델 응답: {response}")
print("\n✅ FP8 모델 정상 작동!")

---
## 📊 실험 결과 기록

| 실험 | SCHEME | 샘플 수 | MaxLen | LB Score | 비고 |
|------|--------|---------|--------|----------|------|
| FP8 기본 | FP8(W8A8) | 512 | 1024 | - | L4 하드웨어 가속 기대 |

---
## ⏭️ 다음 단계
- FP8이 GPTQ보다 LB 점수가 높으면 → FP8을 메인 전략으로 확정
- 성능(정확도)이 너무 낮으면 → `04_qlora_sft.ipynb`로 QLoRA 회복